 # Надо:
* Считать данные из текстового файла
* Переделать в numpy array(2 размерности)
* **ОБЯЗАТЕЛЬНО** нагенерить данных для обучения
* Разбить на выборки
* Обучить модель
* Перевести предсказания в пары

In [9]:
import numpy as np
import pandas as ps

In [20]:
dtt = open('ex.txt', 'r')
dt = dtt.readlines()
dat = [] #Будущие данные о пользователях
for usr in dt:
    dat.append(usr.split(',')) #ВАЖНО! разделитель данных -- ЗАПЯТАЯ

''' Данные о пользователе: [id, nickname, room_number, direction, (sports...)]
                           [0,  kllllevo, 13,           1,         0,1,0,0]
'''
#dat
dtt.close()

In [21]:
data = {}
for usr in dat:
    data[usr[0]] = list(usr[1:])

''' Данные о пользователях: {'id':[Данные о пользователе, кроме id]} 
'''
data

{'\n': [],
 '1': ['Екатерина Самохина', '16', '1', '1', '0', '0\n'],
 '2': ['Даниил Царьков', '6', '2', '0', '0', '0\n'],
 '3': ['Илья Николаев', '6', '2', '0', '0', '0\n'],
 '4': ['Григорий Морозов', '14', '4', '1', '0', '0\n'],
 '5': ['Автономова Ксения', '22', '3', '0', '0', '0\n'],
 '6': ['Пушкарев Святослав', '20', '3', '0', '0', '0\n']}

In [22]:
drata = []
for i in list(data.keys()):
    a = list(data.keys())
    a.remove(i)
    drata.append([val for val in a])

''' Данные вида [another users id] -- this for user with id=0
                [another users id] -- for id=1
                [another users id] -- for id=2
'''
#drata

' Данные вида [another users id] -- this for user with id=0\n                [another users id] -- for id=1\n                [another users id] -- for id=2\n'

In [23]:
final_data = []
nmb = 1 # id юзера(см. ниже)
for lst in drata:
    top_ids= [] #здесь хранятся id
    with_ids = [] # тут -- число совпадений
    top = []
    for u in lst:
        u_data = np.array(data[u][1:])# Вся дата того, кого проверяем, кроме nickname
        usr_data = np.array(data[str(nmb)][1:])# Вся дата того, ДЛЯ кого проверяем, кроме nickname
        with_ids.append(len(np.where(u_data == usr_data)[0]))
        top_ids.append(u)
    args_by_sort = np.argsort(np.array(with_ids))
    #print('top_ids',top_ids)
    #print('with_ids',with_ids)
    #print('args_by_sort',args_by_sort)
    for arg in args_by_sort:
        top.append(top_ids[arg])
    final_data.append(top)
    nmb += 1
'''Вот это б*ть то, ради чего мы все тут собрались. Данная клетка воплощает идею о том,
Чтобы составить все возможные пары киллер-жертва, и отсортировать потенциальных жертв по
кол-ву совпадений по признакам. Выхлоп -- внизу
'''

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  # Remove the CWD from sys.path while we load stuff.


KeyError: '7'

In [6]:
final_data

[['9', '2', '3', '5', '6', '7', '8', '4', '10'],
 ['1', '4', '7', '10', '3', '5', '6', '9', '8'],
 ['1', '6', '8', '10', '2', '4', '5', '7', '9'],
 ['5', '2', '6', '9', '3', '7', '8', '1', '10'],
 ['4', '10', '1', '7', '2', '3', '6', '8', '9'],
 ['3', '4', '8', '10', '1', '2', '5', '9', '7'],
 ['8', '2', '5', '9', '1', '3', '4', '10', '6'],
 ['7', '3', '6', '10', '1', '4', '9', '2', '5'],
 ['1', '10', '4', '7', '2', '6', '8', '3', '5'],
 ['5', '9', '2', '3', '6', '8', '7', '1', '4']]

In [7]:
weighted_data = []
nmb = 1
for lst in final_data:#Не забыть nmb += 1
    weighted = []
    cloned_final = list(final_data)
    cloned_final.remove(lst)
    for u in lst:   
        weight = 0
        for sub_lst in cloned_final:
            for sub_u in sub_lst:
                if sub_lst.index(sub_u) <= lst.index(u) and u == sub_u:
                    weight += lst.index(u) - sub_lst.index(sub_u) + 1
        weighted.append(weight)
    weighted_data.append(weighted)
''' Данная чудо-ячейка возвращает нам матрицу с "весами" id ,где каждому элементу этого массива
сопоставляется id из final_data
'''
weighted_data

[[0, 2, 5, 10, 11, 17, 21, 36, 43],
 [2, 3, 3, 9, 15, 19, 25, 28, 38],
 [2, 0, 4, 9, 17, 21, 26, 32, 37],
 [1, 2, 3, 8, 15, 17, 21, 43, 43],
 [0, 1, 9, 7, 17, 23, 25, 29, 37],
 [0, 3, 4, 9, 20, 25, 26, 28, 41],
 [0, 2, 6, 8, 20, 23, 28, 34, 42],
 [0, 2, 3, 9, 20, 21, 21, 42, 42],
 [2, 1, 7, 7, 17, 17, 21, 40, 42],
 [1, 2, 6, 9, 11, 14, 24, 43, 45]]

In [8]:
%%time
pair = []
for n in range(0, len(final_data)):
    ind = np.argmin(weighted_data[n])
    cdel = int(final_data[n][ind]) - 1
    for j in range(len(final_data[cdel])):
        if final_data[cdel][j] == str(n + 1):
            del weighted_data[cdel][j]
            del final_data[cdel][j]
            break
    pair.append(cdel + 1)
    for i in range(len(final_data)):
        if i != n and i != cdel:
            for j in range(0, len(final_data[i])):
                if final_data[i][j] == str(cdel + 1):
                    del final_data[i][j]
                    del weighted_data[i][j]
                    break
print(pair)

[9, 1, 6, 5, 10, 4, 8, 3, 7, 2]
Wall time: 1.5 ms
